### Define Imports

In [1]:
import create_template_model as CT

In [2]:
imports ="""
import statsmodels.api as sm
import holidays
"""


### Define Class Name

In [3]:
class_name = "SARIMAX"


###  Define Parameters for model

In [4]:

param = "param"

input_params = f"""
    self.parameters_list = kwargs.get('parameters_list',None)
"""

init_function = f"""
def __init__(self,**kwargs):

    #input parameters

    self.train,self.test = kwargs.get('train'),kwargs.get('test')       
    self.random_state = kwargs.get('random_state',1)
    self.target_dates = kwargs.get('target_dates')
    self.target_value = kwargs.get('target_value')
    self.target_items = kwargs.get('target_items')
    {input_params}
    self.n_periods = kwargs.get('n_periods',3)

    #output values
    self.fitted_values = pd.DataFrame()
    self.test_prediction = pd.DataFrame()
    self.unseen_prediction = pd.DataFrame()        
    self.apes = []
    self.run_model()
    del self.train,self.test
"""

### Fit method specific to Model

In [5]:
fit_method = f"""sm.tsa.statespace.SARIMAX(data,exog=np.asarray(exog_data),order=(param[0],param[1],param[2]),
                                            seasonal_order=(param[3],param[4],param[5],param[6])).fit(disp=-1)
"""

fit_function = f"""
def fit(self,data,exog_data,param=None):
        
    ###########
    #  Fit funtion will fit your train data and return model
    ###########
    return {fit_method}
"""

### Define Predict method

In [6]:
# run_script = None
predict_method = "model.predict(exog=exog_data,start=start,end=start+n_periods-1)"
predict_function =f"""
def predict(self,model,exog_data,start,n_periods):
    return {predict_method} 
""" 

### Fitted script

In [7]:
fitted_data = f"model.fittedvalues[param[6]+param[1]:]"
fitted_function =f"""
def fitted_data(self,model,param=None):
    return {fitted_data} 
""" 

### Additional Functions

In [8]:
additional_functions = """
def split_train_test(self,subset_df):
    return subset_df[:-3],subset_df[-3:]

def mean_absolute_percentage_error(self,y_true,y_pred):
    return np.mean(np.abs(np.subtract(y_true,y_pred)/y_true))*100  

def calculate_apes(self):
    for i,j in zip(self.test[self.target_value].values.flatten(),self.test_prediction.values.flatten()):            
        self.apes.append(self.mean_absolute_percentage_error(i,j))

def forecast_exogs(self,val_set):

    max_date = val_set.index.max()
    unseen_dates = pd.DataFrame()
    unseen_dates[self.target_dates] = pd.date_range(max_date+pd.tseries.offsets.MonthEnd(n=1),
              max_date+pd.tseries.offsets.MonthEnd(n=self.n_periods),freq='M')

    start_year = val_set.index.max().year
    end_year = start_year+2
    us_holidays=[]
    for date in holidays.UnitedStates(years=range(start_year,end_year)).items():
        us_holidays.append([str(date[0]),date[1]])

    us_holidays=pd.DataFrame(us_holidays,columns=[self.target_dates,'holiday'])
    us_holidays=pd.DataFrame(us_holidays,columns=[self.target_dates,'holiday'])
    us_holidays[self.target_dates]=pd.to_datetime(us_holidays[self.target_dates])
    us_holidays.holiday=us_holidays.holiday.astype(str)
    us_holidays['minor_holiday']=0
    minor_holidays = ['Martin Luther King, Jr. Day' ,"Washington's Birthday" ,'Columbus Day' ,'Veterans Day'] 
    us_holidays.loc[us_holidays.holiday.isin(minor_holidays),'minor_holiday']=1
    us_holidays['major_holiday']=0
    major_holidays = ["New Year's Day" ,"Christmas Day",'Thanksgiving','Memorial Day','Labor Day','Independence Day'] 
    us_holidays.loc[us_holidays.holiday.isin(major_holidays),'major_holiday']=1
    us_holidays['observed_holiday']=0
    observed =  us_holidays.holiday.apply(lambda row: True if row[1].endswith("(Observed)") else False)
    us_holidays.loc[observed,'observed_holiday']=1
    us_holidays = us_holidays.convert_dtypes()
    us_holidays.sort_values(self.target_dates).reset_index(drop=True,inplace=True)
    us_holidays = us_holidays.set_index(self.target_dates).resample('M').sum().reset_index()

    return pd.merge(unseen_dates,us_holidays,on=self.target_dates,how='left')[['minor_holiday','major_holiday','observed_holiday']]

def optimize_SARIMAX(self,parameters_list,train_set,val_set,exog_columns):

    val_set1 = val_set.copy()
    val_set=val_set[self.target_value].to_numpy()
    results=[]
    best_adj_mape = float('inf')
#         self.train = self.train.astype(float)
    for param in parameters_list:
        try:
            model=self.fit(train_set[self.target_value].astype(float),train_set.loc[:,exog_columns].astype(float).reset_index(drop=True),param)
            fore1=self.predict(model,val_set1.loc[:,exog_columns].astype(float).reset_index(drop=True),self.train.shape[0],self.n_periods)
            fore=np.array(fore1)

            y_true=np.array(list(train_set[param[6]+param[1]:][self.target_value]))
            y_pred=np.array(list(model.fittedvalues[param[6]+param[1]:]))
            train_mape=round((self.mean_absolute_percentage_error(y_true,y_pred)),2)
            val_mape=round((self.mean_absolute_percentage_error(val_set,fore)),2)
            adj_mape = train_mape*len(y_true)/(len(y_true)+len(val_set))+val_mape*len(val_set)/(len(y_true)+len(val_set))
            if adj_mape <= best_adj_mape:
                best_adj_mape=adj_mape
                best_model = model    
            results.append([param,model.aic,train_mape,val_mape,adj_mape,fore])
        except:
            continue

    result_table=pd.DataFrame(results)

    result_table.columns=['parameters','aic','train_mape','val_mape','adj_mape','test_prediction']
    val_results = result_table.sort_values(by='adj_mape',ascending=True).reset_index(drop=True).copy()
    test_predict = val_results.loc[:5]['test_prediction'].apply(lambda row: pd.Series(row).T).mean()

    self.test_prediction = pd.DataFrame(test_predict.values.flatten(),self.test.index,columns=[self.target_value])
    return val_results, best_model

"""

### Running Models and storing results

In [9]:
run_script = f"""
def run_model(self):
    exog_columns =  ['minor_holiday','major_holiday', 'observed_holiday']
    val_results, best_model = self.optimize_SARIMAX(self.parameters_list,self.train,self.test,exog_columns)
    #Model running for test prediction
    fitted_val_list=[]

    actual_data = self.train.append(self.test)
    for param in val_results['parameters'].loc[:5]:
        try:

            model = self.fit(actual_data[self.target_value],actual_data.loc[:,exog_columns].astype(float).reset_index(drop=True),param)
            fore_test=self.predict(model,self.forecast_exogs(self.test).astype(float).reset_index(drop=True),actual_data.shape[0],self.n_periods)
            fitted_val_list.append(fore_test.values.flatten())

        except:
            continue

    fitted_val=pd.DataFrame(fitted_val_list,columns=np.arange(1,self.n_periods+1))
    fitted_mean=[]
    for i in np.arange(1,self.n_periods+1):
        fitted_mean.append(fitted_val[i].mean())

    unseen_index = pd.date_range(self.test.index[-1]+pd.tseries.offsets.MonthEnd(n=1),
          self.test.index[-1]+pd.tseries.offsets.MonthEnd(n=self.n_periods),freq='M')

    self.unseen_prediction = pd.DataFrame(fitted_mean,unseen_index,columns=[self.target_value])

    #Calculate apes for each test prediction        
    self.calculate_apes()
    self.train_model_params = val_results['parameters'].loc[:5]            
    self.test_model_params = val_results['parameters'].loc[:5]
    self.train_mape = val_results['train_mape'].loc[:5].mean()  
    self.test_mape = val_results['val_mape'].loc[:5].mean()

"""

In [10]:
template_dict = dict(imports=imports,class_name=class_name,init_function=init_function,
                         fit_function=fit_function,fitted_function=fitted_function,predict_function=predict_function,
                        additional_functions=additional_functions,run_script=run_script)

In [11]:
CT.render_template(template_dict=template_dict)

In [12]:
print(CT.get_ouput())

#Import files


import statsmodels.api as sm
import holidays

import pandas as pd
import numpy as np

#Model class
class SARIMAX:

  

    def __init__(self,**kwargs):

        #input parameters

        self.train,self.test = kwargs.get('train'),kwargs.get('test')       
        self.random_state = kwargs.get('random_state',1)
        self.target_dates = kwargs.get('target_dates')
        self.target_value = kwargs.get('target_value')
        self.target_items = kwargs.get('target_items')
        
        self.parameters_list = kwargs.get('parameters_list',None)

        self.n_periods = kwargs.get('n_periods',3)

        #output values
        self.fitted_values = pd.DataFrame()
        self.test_prediction = pd.DataFrame()
        self.unseen_prediction = pd.DataFrame()        
        self.apes = []
        self.run_model()
        del self.train,self.test



    def fit(self,data,exog_data,param=None):
            
        ###########
        #  Fit funtion will fit your train dat

In [13]:
class_name

'SARIMAX'

In [14]:
from model.SARIMAX import SARIMAX
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.options.mode.chained_assignment = None

c:\users\ck073783\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [15]:
import json

with open('model-config.json') as f:
  config = json.load(f)

In [16]:
config.keys()


dict_keys(['random_state', 'n_periodss', 'not_include_be_list', 'overall_be', 'aggregate_choice', 'target_file', 'target_dates', 'target_value', 'target_items', 'other_target_value', 'raw_data_path', 'processed_file'])

In [17]:
import os
import pickle
import pandas as pd

target_dates = 'posted_date'
target_values = 'total_charge'
target_items = 'billing_entity'
transformed_data_path = r'C:\Users\CK073783\OneDrive - Cerner Corporation\Desktop\Projects\Revenue Forecasting\Versions\version 2\data\processed_data\All Adventist W\\'

data_files = os.listdir(transformed_data_path)
model_output_path = r'C:\Users\CK073783\OneDrive - Cerner Corporation\Desktop\Projects\Revenue Forecasting\Versions\version 2\data\model_output\\'

def filter_data(summary_df):
        return summary_df[(summary_df['valid'])&(summary_df['months_missing']==0)][target_items]

filename = data_files[0]
filehandler = open(transformed_data_path+filename, 'rb') 
data_object = pickle.load(filehandler)
data_date = data_object.summary_df.end_date.max().month_name() +'-'+str(data_object.summary_df.end_date.max().year)

filtered_data  = data_object.transform_data[data_object.transform_data[target_items].isin(filter_data(data_object.summary_df))]
BE = 'All Adventist W'
subset_df = filtered_data[filtered_data[target_items]==BE].reset_index(drop=True)
subset_df[target_dates] = pd.to_datetime(subset_df[target_dates]).copy()
subset_df.set_index('posted_date',inplace=True)
# models = Univariate(BE,data = subset_df[target_value],target_items= target_items,target_value=target_value,target_dates=target_dates)
subset_df.head()

,billing_entity,total_charge,count,minor_holiday,major_holiday,observed_holiday
posted_date,,,,,,
2015-01-31,All Adventist W,7.709414e+08,213504.0,1,1,0
2015-02-28,All Adventist W,8.748842e+08,241981.0,1,0,0
2015-03-31,All Adventist W,8.972711e+08,266676.0,0,0,0
2015-04-30,All Adventist W,8.930693e+08,268530.0,0,0,0
2015-05-31,All Adventist W,8.659224e+08,261408.0,0,1,0


In [18]:
from itertools import product
def split_train_test(subset_df):
        return subset_df[:-3],subset_df[-3:]
train,test = split_train_test(subset_df)
p=range(0,2)
d=range(0,1)
q=range(0,2)
P=range(0,2)
D=range(0,1)
Q=range(0,2)
s=(3,6,12)

parameters=product(p,d,q,P,D,Q,s)
parameters_list=list(parameters)

# use_arma_errors=[True,False]
# use_box_cox=[True,False]
# use_trend=[True,False]
# use_damped_trend=[True,False]  

# parameters=product(use_arma_errors,use_box_cox,use_trend,use_damped_trend)
# parameters_list=list(parameters)

def check(**kwargs):
    print(kwargs.keys())
config['parameters_list'] = parameters_list
config['train'] = train
config['test'] = test
check(**config)
obj = SARIMAX(**config)

dict_keys(['random_state', 'n_periodss', 'not_include_be_list', 'overall_be', 'aggregate_choice', 'target_file', 'target_dates', 'target_value', 'target_items', 'other_target_value', 'raw_data_path', 'processed_file', 'parameters_list', 'train', 'test'])


In [19]:
obj.apes

[14.003346609577044, 5.2054298437018325, 16.9135002366496]

In [ ]:
obj.unseen_prediction